In [1]:
import requests
import json
import pymongo
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import gensim
# from bson.objectid import ObjectId

# Connect to db

In [2]:
f = open("password.txt", "r")
pwd =  f.read()
f.close()

In [3]:
db_username = 'sveta'
db_password = pwd

In [4]:
db = MongoClient("mongodb+srv://" + db_username + ":" + db_password + "@makhela-qvsh8.mongodb.net/Makhela")

In [5]:
mydb = db["Makhela"]

In [6]:
mydb.list_collection_names()

['blacklist',
 'keywords',
 'settings',
 'posts',
 'opinion_leaders',
 'suggestions',
 'users']

# Opinion Leaders

In [7]:
mycol = mydb["opinion_leaders"].find()

In [8]:
leaders = {}

for leader in mycol:
    try:
        leaders[leader["twitter_id"]] = leader
        leader["posts"] = []
#         print(leader["twitter_id"])
    except:
        continue

In [10]:
leaders[1589253019]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3ba1'),
 'full_name': 'Richard (Dick) Sokolsky',
 'new_leader': False,
 'twitter_created_at': datetime.datetime(2013, 7, 12, 19, 28, 53),
 'twitter_description': '',
 'twitter_followers_count': 55,
 'twitter_friends_count': 57,
 'twitter_id': 1589253019,
 'twitter_location': '',
 'twitter_screen_name': 'dicktfirefly',
 'twitter_statuses_count': 5,
 'level_of_certainty': 10,
 'community_following': [228136993, 22843167],
 'lock': False,
 'twitter_profile_image': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_400x400.png',
 'posts': []}

# Posts

In [9]:
mycol = mydb["posts"].find()

In [10]:
posts = {}

for post in mycol:
    try:
        posts[post["post_id"]] = post
    except:
        continue

In [11]:
posts[1235946337331343366]

{'_id': ObjectId('5e6407df124e74ea22f2a87e'),
 'leader_twitter_id': 2696666707,
 'post_id': 1235946337331343366,
 'full_text': '@michaeltanchum Yep, this applies globally. The need for truth, data and transparency as well.',
 'date_created': datetime.datetime(2020, 3, 6, 15, 12, 24),
 'in_reply_to_status_id': 1235945439180054528,
 'in_reply_to_status_text': '@dassakaye Exactly. Never has made it more clear in the U.S. that good governance is a matter of life and death.',
 'in_reply_to_status_user_id': 'michaeltanchum',
 'quoted_status_id': 'None',
 'quoted_status_text': 'None',
 'quoted_status_user_id': 'None',
 'retweeted_status_id': 'None',
 'retweeted_status_text': 'None',
 'retweeted_status_user_id': 'None',
 'checked_for_suggestions': True}

In [12]:
def text(dict):
    if dict['retweeted_status_text'] != 'None':
        return dict['retweeted_status_text']
    elif dict['in_reply_to_status_text'] != 'None':
        return dict['in_reply_to_status_text']
    elif dict['quoted_status_text'] != 'None':
        return dict['quoted_status_text']
    else:
        return dict['full_text']

In [13]:
print(text(posts[1235944924853317632]))

LEBANON #Coronavirus Crisis:  #COVID19 spreading in a country that has not been able to buy medical supplies for 5 months. Now Lebanon's health minister says the country is no longer focused on containing the virus https://t.co/NLSSH1PSMT


In [14]:
words_from_posts = []
for key, value in posts.items():
    try:
        post_row = []
        post_row.append(key)
        post_row.append(text(posts[key])) 
#         words_from_posts.append(text(posts[key])) 
        words_from_posts.append(post_row)
    except:
        continue

In [15]:
pr = pd.DataFrame(words_from_posts)
pr.head()

,0,1
0,1235946337331343366,@dassakaye Exactly. Never has made it more cle...
1,1235944924853317632,LEBANON #Coronavirus Crisis: #COVID19 spreadi...
2,1235938139119562754,A further isolated and weakened Iran will like...
3,1235161759423614977,"Coronavirus: Iran temporarily frees 54,000 pri..."
4,1234901226807939072,@dassakaye Not necessarily to be honest. #Saud...


In [16]:
pr[1].head()

0    @dassakaye Exactly. Never has made it more cle...
1    LEBANON #Coronavirus Crisis:  #COVID19 spreadi...
2    A further isolated and weakened Iran will like...
3    Coronavirus: Iran temporarily frees 54,000 pri...
4    @dassakaye Not necessarily to be honest. #Saud...
Name: 1, dtype: object

In [17]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(pr[1])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [18]:
df.head()

,00,000,004,008,00am,00hmvxw0iy,00q3870mh1,01,010,0120kg,...,全く同感なのだが,同時にトランプ政権はきっとこの議論を理解しないだろうとも確信している,新型コロナウィルスは米国の対イラン政策の転換のチャンスという記事とその紹介連ツイ,欧州が同航空機の空域入域を拒否したからとのこと,동해상으로,미상,발사,발사체,북한,합참
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.148258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
len(posts)

6323

In [20]:
df.iloc[0,:].max()

0.3970979024425033

In [21]:
words = []
for i in range(len(df)):
    words.append(df.iloc[i,:][df.iloc[i,:]==df.iloc[i,:].max()].index.values.tolist())

In [22]:
len(words)

6323

In [23]:
pr['words'] = words

In [24]:
pr.head()

,0,1,words
0,1235946337331343366,@dassakaye Exactly. Never has made it more cle...,[dassakaye]
1,1235944924853317632,LEBANON #Coronavirus Crisis: #COVID19 spreadi...,[lebanon]
2,1235938139119562754,A further isolated and weakened Iran will like...,"[emboldening, pscktckw8m]"
3,1235161759423614977,"Coronavirus: Iran temporarily frees 54,000 pri...",[ixktbh9qen]
4,1234901226807939072,@dassakaye Not necessarily to be honest. #Saud...,"[managem, necessarily, ph93rk6s04]"


In [25]:
for i in range(len(pr)):
    try:
        posts[pr.loc[i,0]]['words'] = pr.loc[i,'words']
    except:
        continue


# Add posts to opinion leader

In [26]:
for key, value in posts.items():
    try:
        leaders[value['leader_twitter_id']]["posts"].append(key)
    except:
        continue

In [27]:
# leaders[2696666707]

# LDA

In [28]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [29]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [30]:
import spacy

In [31]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [33]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [34]:
def LDA(posts_corpus):
    df = pd.DataFrame(posts_corpus) 
    df = df.drop_duplicates()
    df = df.rename({0: 'leader', 1: 'post'}, axis='columns')
    
    data = df.post.values.tolist()
    data = [re.sub('\s+', ' ', sent) for sent in data]
    data = [re.sub("\'", "", sent) for sent in data]
    data_words = list(sent_to_words(data))
    
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)
  
    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])


    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=5, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
    return lda_model

In [35]:
for key, value in leaders.items():
    posts_corpus = []
    try:   
        for post in value['posts']:
            leader_posts = []
            leader_posts.append(key)
            leader_posts.append(text(posts[post]))
            posts_corpus.append(leader_posts)
        leaders[key]['topics'] = LDA(posts_corpus).print_topics()
    except:
        continue

In [36]:
leaders[2696666707]

{'_id': ObjectId('5e7b646b59c226581da83177'),
 'twitter_id': 2696666707,
 'full_name': 'Dalia Dassa Kaye',
 'twitter_screen_name': 'dassakaye',
 'twitter_location': 'Santa Monica, CA',
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3492,
 'twitter_friends_count': 404,
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_statuses_count': 3738,
 'new_leader': False,
 'level_of_certainty': 10,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'lock': False,
 'community_following': [3310505366,
  3310505366,
  921728354,
  921728354,
  1199136998,
  3048160253,
  3048160253,
  3048160253,
  17937362,
  130549887,
  30181014,
  2759399473,
  331907658,
  331907658,
  2190601093,
  51241574,
  1624071703,
  482120066,
  482120066,
  2739262472,
  316724801,
  376435807,
  376435807,
  49702592,
  480678403,
  1114663532,

In [37]:
# Compute Perplexity
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

In [38]:

# Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

# Neo4j

In [ ]:
# len(G)

In [ ]:
# G.nodes

In [ ]:
# list(G.nodes(data=True))

In [ ]:
# list(G.edges())

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# Gr = nx.cycle_graph(G)
# pos = nx.circular_layout(Gr)

# # nx.draw_networkx_nodes(G, pos, node_size=150, node_color='blue', alpha=0.3)
# # nx.draw_networkx_edges(G, pos, width=2, alpha=0.3, edge_color='green')
# # nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif')
# # # # default
# # plt.figure(1)
# # smaller nodes and fonts
# # plt.figure(2)
# # larger figure size
# # nx.draw(G,pos,node_size=60,font_size=8) 
# plt.figure(figsize=(12,12)) 
# nx.draw(G,pos,node_size=60)
# plt.show()
# # # plt.savefig('data.png')  

In [ ]:
# print(nx.is_strongly_connected(G))

In [ ]:
# isolated = nx.isolates(G)

In [ ]:
# for i in isolated:
#     G.remove_node(i)

In [ ]:
# nx.center(G)

In [ ]:
# nx.periphery(G)

In [ ]:
# degree_centrality = nx.degree_centrality(G)

In [ ]:
# degree_centrality

In [ ]:
# centrality_values = []
# for key, value in degree_centrality.items():
#     centrality_values.append([key, value])

In [ ]:
# centrality_values

In [ ]:
# centrality_values.sort(key=lambda x: x[1], reverse=True)

In [ ]:
# centrality_values

In [ ]:
# for leader in centrality_values[:5]:
#     print(leaders[leader[0]])

In [ ]:
# for leader in centrality_values[-5:]:
#     print(leaders[leader[0]])

# Graph Neo4j

In [50]:
from py2neo import Graph, Node, Relationship
# from bottle import get, run, request, response, static_file

In [51]:
graph = Graph(password = pwd)

In [41]:
for key, value in leaders.items():
    graph.create(Node("Leader",
                id=key,
                full_name=value['full_name'],
                twitter_screen_name=value['twitter_screen_name'],
                twitter_location=value['twitter_location'],
                twitter_followers_count=value['twitter_followers_count'],
                twitter_friends_count=value['twitter_friends_count'],
                twitter_created_at=value["twitter_created_at"],
                twitter_statuses_count=value['twitter_statuses_count'],
                twitter_profile_image=value['twitter_profile_image'],
                twitter_description=value["twitter_description"]))

In [42]:
for key, value in leaders.items():
    try:
        existing_u1 = graph.nodes.match("Leader", id=key).first()
        for follower in value["community_following"]:
            existing_u2 = graph.nodes.match("Leader", id=follower).first()
            try:
                graph.create(Relationship(existing_u1, 'FOLLOWS', existing_u2))
            except:
                print(key, follower)
    except:
        continue

    

64725597 2190601093
64725597 2190601093
64725597 61051322
64725597 61051322
64725597 1485394590
64725597 1485394590
64725597 376435807
64725597 376435807
43380129 950571448850157568
43380129 348306517
43380129 2739262472
43380129 475454826
43380129 826557130916380673
43380129 48906364
43380129 22803302
43380129 1387074108
43380129 61051322
43380129 376435807
57550888 257442762
57550888 1387074108
57550888 154619001
57550888 376435807
57550888 1114663532
57550888 42176166
57550888 475454826
288869649 348306517
288869649 475454826
288869649 61051322
288869649 1227326168
288869649 42176166
288869649 376435807
288869649 48906364
1151625187 376435807
1151625187 376435807
910253568397053952 2190601093
910253568397053952 61051322
910253568397053952 61051322
910253568397053952 376435807
910253568397053952 376435807
910253568397053952 1114663532
910253568397053952 1114663532
480678403 2739262472
480678403 2190601093
480678403 1485394590
480678403 1485394590
480678403 376435807
480678403 3764358

In [43]:
posts[1235944924853317632]

{'_id': ObjectId('5e6407e0124e74ea22f2a87f'),
 'leader_twitter_id': 2696666707,
 'post_id': 1235944924853317632,
 'full_text': 'Not surprising that countries in MENA region with poor governance and multiple domestic and regional crises may not be able to contain #CoronaVirusChallenge and are now focused on mitigation. https://t.co/xCOHRuFpLl',
 'date_created': datetime.datetime(2020, 3, 6, 15, 6, 47),
 'in_reply_to_status_id': None,
 'in_reply_to_status_text': 'None',
 'in_reply_to_status_user_id': 'None',
 'quoted_status_id': 1235931273073483777,
 'quoted_status_text': "LEBANON #Coronavirus Crisis:  #COVID19 spreading in a country that has not been able to buy medical supplies for 5 months. Now Lebanon's health minister says the country is no longer focused on containing the virus https://t.co/NLSSH1PSMT",
 'quoted_status_user_id': 'michaeltanchum',
 'retweeted_status_id': 'None',
 'retweeted_status_text': 'None',
 'retweeted_status_user_id': 'None',
 'checked_for_suggestions': True,


In [44]:
for key, value in posts.items():
    graph.create(Node("Post",
                id=key,
                full_text=value['full_text'],
                date_created=value['date_created'],
                words=value["words"]))

In [45]:
for key, value in leaders.items():
    try:
        existing_u1 = graph.nodes.match("Leader", id=key).first()
        for post in value["posts"]:
            existing_u2 = graph.nodes.match("Post", id=post).first()
            try:
                graph.create(Relationship(existing_u1, 'TWEETED', existing_u2))
            except:
                print(key, post)
    except:
        continue


In [ ]:
# posts[1236394102721146880]

In [ ]:
# posts[1236380467051540485]

In [46]:
for key, value in posts.items():
    try:
        existing_u1 = graph.nodes.match("Leader", id=value['leader_twitter_id']).first()
        existing_u2 = graph.nodes.match("Post", id=value['retweeted_status_id']).first()
        try:
            graph.create(Relationship(existing_u1, 'RETWEETED', existing_u2))
        except:
            print(existing_u1, existing_u2)
    except:
        continue


(_17:Leader {full_name: 'Dalia Dassa Kaye', id: 2696666707, twitter_created_at: datetime('2014-07-31T21:24:12.000000000'), twitter_description: 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation', twitter_followers_count: 3492, twitter_friends_count: 404, twitter_location: 'Santa Monica, CA', twitter_profile_image: 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg', twitter_screen_name: 'dassakaye', twitter_statuses_count: 3738}) None
(_17:Leader {full_name: 'Dalia Dassa Kaye', id: 2696666707, twitter_created_at: datetime('2014-07-31T21:24:12.000000000'), twitter_description: 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation', twitter_followers_count: 3492, twitter_friends_count: 404, twitter_location: 'Santa Monica, CA', twitter_profile_image: 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg', twitter_screen_name: 'dassakaye', twitter_statuses_count: 3738}) N

(_0:Leader {full_name: 'Richard (Dick) Sokolsky', id: 1589253019, twitter_created_at: datetime('2013-07-12T19:28:53.000000000'), twitter_description: '', twitter_followers_count: 55, twitter_friends_count: 57, twitter_location: '', twitter_profile_image: 'http://abs.twimg.com/sticky/default_profile_images/default_profile_400x400.png', twitter_screen_name: 'dicktfirefly', twitter_statuses_count: 5}) None
(_0:Leader {full_name: 'Richard (Dick) Sokolsky', id: 1589253019, twitter_created_at: datetime('2013-07-12T19:28:53.000000000'), twitter_description: '', twitter_followers_count: 55, twitter_friends_count: 57, twitter_location: '', twitter_profile_image: 'http://abs.twimg.com/sticky/default_profile_images/default_profile_400x400.png', twitter_screen_name: 'dicktfirefly', twitter_statuses_count: 5}) None
(_0:Leader {full_name: 'Richard (Dick) Sokolsky', id: 1589253019, twitter_created_at: datetime('2013-07-12T19:28:53.000000000'), twitter_description: '', twitter_followers_count: 55, twi

(_14:Leader {full_name: 'Michael Elleman', id: 910253568397053952, twitter_created_at: datetime('2017-09-19T21:25:37.000000000'), twitter_description: 'Director, Non-Proliferation and Nuclear Policy at the International Institute for Strategic Studies. Expressed opinions are mine, re-tweets are not endorsements', twitter_followers_count: 3603, twitter_friends_count: 1943, twitter_location: 'Washington, DC', twitter_profile_image: 'http://pbs.twimg.com/profile_images/1040279155890376704/6utHF6RS_400x400.jpg', twitter_screen_name: 'EllemanIISS', twitter_statuses_count: 2499}) None
(_14:Leader {full_name: 'Michael Elleman', id: 910253568397053952, twitter_created_at: datetime('2017-09-19T21:25:37.000000000'), twitter_description: 'Director, Non-Proliferation and Nuclear Policy at the International Institute for Strategic Studies. Expressed opinions are mine, re-tweets are not endorsements', twitter_followers_count: 3603, twitter_friends_count: 1943, twitter_location: 'Washington, DC', twi

None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
None None
(_2:Leader {full_name: 'Raz Zimmt', id: 1515812534, twitter_created_at: datetime('2013-06-14T08:47:11.000000000'), twitter_description: 'Tel Aviv University, INSS, ITIC.  https://t.co/OzzbJA2b4U', twitter_followers_count: 12506, twitter_friends_count: 2875, twitter_location: '', twitter_profile_image: 'http://pbs.twimg.com/profile_images/1043857056401432576/_zRXKejD_400x400.jpg', twitter_screen_name: 'RZimmt', twitter_statuses_count: 17524}) None
(_2:Leader {full_name: 'Raz Zimmt', id: 1515812534, twitter_created_at: datetime('2013-06-14T08:47:11.000000000'), twitter_description: 'Tel Aviv University, INSS, ITIC.  https://t.co/OzzbJA2b4U', twitter_followers_count: 12506, twitter_friends_count: 2875, twitter_loc

(_75:Leader {full_name: 'Esfandyar Batmanghelidj', id: 2759399473, twitter_created_at: datetime('2014-08-23T16:28:47.000000000'), twitter_description: 'I work on business diplomacy. Mostly #Iran, also #Uzbekistan. Founder of @BourseBazaar + @EurasianInvstr. Columnist at @bopinion.', twitter_followers_count: 11448, twitter_friends_count: 1672, twitter_location: '', twitter_profile_image: 'http://pbs.twimg.com/profile_images/751338911587246080/yUjalZbe_400x400.jpg', twitter_screen_name: 'yarbatman', twitter_statuses_count: 19848}) None
(_75:Leader {full_name: 'Esfandyar Batmanghelidj', id: 2759399473, twitter_created_at: datetime('2014-08-23T16:28:47.000000000'), twitter_description: 'I work on business diplomacy. Mostly #Iran, also #Uzbekistan. Founder of @BourseBazaar + @EurasianInvstr. Columnist at @bopinion.', twitter_followers_count: 11448, twitter_friends_count: 1672, twitter_location: '', twitter_profile_image: 'http://pbs.twimg.com/profile_images/751338911587246080/yUjalZbe_400x4

...

(_14:Leader {full_name: 'Michael Elleman', id: 910253568397053952, twitter_created_at: datetime('2017-09-19T21:25:37.000000000'), twitter_description: 'Director, Non-Proliferation and Nuclear Policy at the International Institute for Strategic Studies. Expressed opinions are mine, re-tweets are not endorsements', twitter_followers_count: 3603, twitter_friends_count: 1943, twitter_location: 'Washington, DC', twitter_profile_image: 'http://pbs.twimg.com/profile_images/1040279155890376704/6utHF6RS_400x400.jpg', twitter_screen_name: 'EllemanIISS', twitter_statuses_count: 2499}) None
(_14:Leader {full_name: 'Michael Elleman', id: 910253568397053952, twitter_created_at: datetime('2017-09-19T21:25:37.000000000'), twitter_description: 'Director, Non-Proliferation and Nuclear Policy at the International Institute for Strategic Studies. Expressed opinions are mine, re-tweets are not endorsements', twitter_followers_count: 3603, twitter_friends_count: 1943, twitter_location: 'Washington, DC', twi

(_24:Leader {full_name: 'Farid Senzai', id: 1624071703, twitter_created_at: datetime('2013-07-26T22:57:58.000000000'), twitter_description: 'Founder & President (emeritus) @CGlobalPolicy - \nProfessor Middle East Politics at SCU - Ph.D. Oxford Univ. - \nAuthor: Political Islam in Age of Democratization', twitter_followers_count: 10151, twitter_friends_count: 1904, twitter_location: 'San Francisco - Washington DC', twitter_profile_image: 'http://pbs.twimg.com/profile_images/378800000399004156/2efc475d05ec8d41410f987b52cf7edc_400x400.jpeg', twitter_screen_name: 'Farid_Senzai', twitter_statuses_count: 6363}) None
(_25:Leader {full_name: 'Jackie Calmes', id: 146167871, twitter_created_at: datetime('2010-05-20T19:22:34.000000000'), twitter_description: 'Editor, LA Times DC bureau. Ex NYT & WSJ, covering White House, Congress, politics & policy for 3+ decades. Ex of TX, OH', twitter_followers_count: 17585, twitter_friends_count: 1135, twitter_location: 'Washington, D.C.', twitter_profile_ima

(_74:Leader {full_name: 'Jason Brodsky', id: 1908627954, twitter_created_at: datetime('2013-09-26T17:41:38.000000000'), twitter_description: 'Policy Director @UANI, formerly with @TheWilsonCenter & @WhiteHouse, #Iran wonk, foreign policy analyst, and speechwriter. Views are my own.', twitter_followers_count: 3605, twitter_friends_count: 5001, twitter_location: 'New York, NY', twitter_profile_image: 'http://pbs.twimg.com/profile_images/1234633499723943936/Fny0BEyI_400x400.jpg', twitter_screen_name: 'JasonMBrodsky', twitter_statuses_count: 16263}) None
(_74:Leader {full_name: 'Jason Brodsky', id: 1908627954, twitter_created_at: datetime('2013-09-26T17:41:38.000000000'), twitter_description: 'Policy Director @UANI, formerly with @TheWilsonCenter & @WhiteHouse, #Iran wonk, foreign policy analyst, and speechwriter. Views are my own.', twitter_followers_count: 3605, twitter_friends_count: 5001, twitter_location: 'New York, NY', twitter_profile_image: 'http://pbs.twimg.com/profile_images/1234

(_72:Leader {full_name: 'Vipin Narang', id: 1707281120, twitter_created_at: datetime('2013-08-28T13:16:43.000000000'), twitter_description: 'MIT Professor, Carnegie Nonresident Scholar, chemical engineer long ago, studies nuclear proliferation and strategy, also some South Asia.', twitter_followers_count: 38150, twitter_friends_count: 1064, twitter_location: 'Cambridge, Massachusetts', twitter_profile_image: 'http://pbs.twimg.com/profile_images/1152646721241006084/AmUIM2o3_400x400.jpg', twitter_screen_name: 'NarangVipin', twitter_statuses_count: 1861}) None
(_72:Leader {full_name: 'Vipin Narang', id: 1707281120, twitter_created_at: datetime('2013-08-28T13:16:43.000000000'), twitter_description: 'MIT Professor, Carnegie Nonresident Scholar, chemical engineer long ago, studies nuclear proliferation and strategy, also some South Asia.', twitter_followers_count: 38150, twitter_friends_count: 1064, twitter_location: 'Cambridge, Massachusetts', twitter_profile_image: 'http://pbs.twimg.com/pr

(_54:Leader {full_name: 'Angry Staffer', id: 848148994102611969, twitter_created_at: datetime('2017-04-01T12:24:12.000000000'), twitter_description: 'Not a current WH Staffer | Politics, NatSec, and Snark \u2014 Mileage may vary | Views mine |', twitter_followers_count: 209993, twitter_friends_count: 373, twitter_location: '', twitter_profile_image: 'http://pbs.twimg.com/profile_images/1056018749604663297/3KdM71fN_400x400.jpg', twitter_screen_name: 'AngrierWHStaff', twitter_statuses_count: 20572}) None
(_56:Leader {full_name: 'Eli Clifton', id: 63895732, twitter_created_at: datetime('2009-08-08T03:29:19.000000000'), twitter_description: 'Now @QuincyInst. I report on money in politics.', twitter_followers_count: 8836, twitter_friends_count: 2309, twitter_location: 'New York', twitter_profile_image: 'http://pbs.twimg.com/profile_images/515200246846992384/_z5CBGbZ_400x400.jpeg', twitter_screen_name: 'EliClifton', twitter_statuses_count: 22964}) None
(_56:Leader {full_name: 'Eli Clifton', 

In [47]:
# graph.run("MATCH (a:Leader) RETURN a.full_name, a.twitter_description").data()

[{'a.full_name': 'Richard (Dick) Sokolsky', 'a.twitter_description': ''},
 {'a.full_name': 'Karim Sadjadpour',
  'a.twitter_description': 'Senior Fellow @CarnegieEndow; Currently writing a book for @AAKnopf; Adjunct professor @Georgetown; Former YGL @WEF; Board member @ICHRI'},
 {'a.full_name': 'Raz Zimmt',
  'a.twitter_description': 'Tel Aviv University, INSS, ITIC.  https://t.co/OzzbJA2b4U'},
 {'a.full_name': 'Ahmad Majidyar',
  'a.twitter_description': 'Middle East analyst, with a focus on Iran and Afghanistan. Formerly, MEI, AEI, BBC & UN. Statements here = my own views; RTs & Likes≠Endorsement.'},
 {'a.full_name': 'Norman T. Roule',
  'a.twitter_description': '@thecipherbrief expert; @FightExtremism & @UANI adviser   Retweet or Like do not equal endorsement.  Views are my own only.'},
 {'a.full_name': 'Pierre Goldschmidt', 'a.twitter_description': ''},
 {'a.full_name': 'Reuel Mark Gerecht',
  'a.twitter_description': 'Author and @FDD Senior Fellow focusing on Iran, Iraq, Afghanist

In [55]:
graph.run("CALL algo.pageRank('Leader', 'FOLLOWS', {iterations:20, dampingFactor:0.85, write: true,writeProperty:'pagerank'}) YIELD nodes, iterations, loadMillis, computeMillis, writeMillis, dampingFactor, write, writeProperty")

In [57]:
graph.run("MATCH (a:Leader) RETURN a.full_name, a.pagerank").data()

[{'a.full_name': 'Richard (Dick) Sokolsky', 'a.pagerank': 0.15000000000000002},
 {'a.full_name': 'Karim Sadjadpour', 'a.pagerank': 0.18509538714517157},
 {'a.full_name': 'Raz Zimmt', 'a.pagerank': 0.164405402053841},
 {'a.full_name': 'Ahmad Majidyar', 'a.pagerank': 0.16430282546346836},
 {'a.full_name': 'Norman T. Roule', 'a.pagerank': 0.16947532151411868},
 {'a.full_name': 'Pierre Goldschmidt', 'a.pagerank': 0.15000000000000002},
 {'a.full_name': 'Reuel Mark Gerecht', 'a.pagerank': 0.15000000000000002},
 {'a.full_name': 'Uzi Rubin', 'a.pagerank': 0.15000000000000002},
 {'a.full_name': 'Ariane M. Tabatabai', 'a.pagerank': 0.18509538714517157},
 {'a.full_name': 'Matthew Kroenig', 'a.pagerank': 0.16430282546346836},
 {'a.full_name': 'Afshon Ostovar', 'a.pagerank': 0.16947532151411868},
 {'a.full_name': 'Gary Samore', 'a.pagerank': 0.15000000000000002},
 {'a.full_name': 'Ray Takeyh', 'a.pagerank': 0.15000000000000002},
 {'a.full_name': 'Mark Hibbs', 'a.pagerank': 0.20556200568809874},
 {'

In [58]:
graph.run("CALL algo.betweenness('Leader','FOLLOWS', {direction:'out',write:true, writeProperty:'centrality'}) YIELD nodes, minCentrality, maxCentrality, sumCentrality, loadMillis, computeMillis, writeMillis")

In [59]:
graph.run("MATCH (a:Leader) RETURN a.full_name, a.centrality").data()

[{'a.full_name': 'Richard (Dick) Sokolsky', 'a.centrality': 0.0},
 {'a.full_name': 'Karim Sadjadpour', 'a.centrality': 4.352777777777778},
 {'a.full_name': 'Raz Zimmt', 'a.centrality': 1.861111111111111},
 {'a.full_name': 'Ahmad Majidyar', 'a.centrality': 7.561904761904763},
 {'a.full_name': 'Norman T. Roule', 'a.centrality': 1.5416666666666665},
 {'a.full_name': 'Pierre Goldschmidt', 'a.centrality': 0.0},
 {'a.full_name': 'Reuel Mark Gerecht', 'a.centrality': 0.0},
 {'a.full_name': 'Uzi Rubin', 'a.centrality': 0.0},
 {'a.full_name': 'Ariane M. Tabatabai', 'a.centrality': 9.165811965811967},
 {'a.full_name': 'Matthew Kroenig', 'a.centrality': 0.5333333333333333},
 {'a.full_name': 'Afshon Ostovar', 'a.centrality': 4.536446886446886},
 {'a.full_name': 'Gary Samore', 'a.centrality': 0.0},
 {'a.full_name': 'Ray Takeyh', 'a.centrality': 0.0},
 {'a.full_name': 'Mark Hibbs', 'a.centrality': 32.997488984445496},
 {'a.full_name': 'Michael Elleman', 'a.centrality': 71.87392509451334},
 {'a.full_

In [61]:
graph.run("CALL algo.closeness('Leader', 'FOLLOWS', {write:true, writeProperty:'centrality_closeness'}) YIELD nodes,loadMillis, computeMillis, writeMillis")

In [62]:
graph.run("MATCH (a:Leader) RETURN a.full_name, a.centrality_closeness").data()

[{'a.full_name': 'Richard (Dick) Sokolsky', 'a.centrality_closeness': 0.0},
 {'a.full_name': 'Karim Sadjadpour',
  'a.centrality_closeness': 0.44805194805194803},
 {'a.full_name': 'Raz Zimmt', 'a.centrality_closeness': 0.41566265060240964},
 {'a.full_name': 'Ahmad Majidyar',
  'a.centrality_closeness': 0.6160714285714286},
 {'a.full_name': 'Norman T. Roule',
  'a.centrality_closeness': 0.4394904458598726},
 {'a.full_name': 'Pierre Goldschmidt',
  'a.centrality_closeness': 0.3854748603351955},
 {'a.full_name': 'Reuel Mark Gerecht', 'a.centrality_closeness': 0.0},
 {'a.full_name': 'Uzi Rubin', 'a.centrality_closeness': 0.0},
 {'a.full_name': 'Ariane M. Tabatabai',
  'a.centrality_closeness': 0.5149253731343284},
 {'a.full_name': 'Matthew Kroenig',
  'a.centrality_closeness': 0.5073529411764706},
 {'a.full_name': 'Afshon Ostovar',
  'a.centrality_closeness': 0.5111111111111111},
 {'a.full_name': 'Gary Samore', 'a.centrality_closeness': 0.0},
 {'a.full_name': 'Ray Takeyh', 'a.centrality_cl

In [66]:
graph.run("CALL algo.eigenvector('Leader', 'FOLLOWS', {write: true, writeProperty:'eigenvector'}) YIELD nodes, iterations, loadMillis, computeMillis, writeMillis, dampingFactor, write, writeProperty")

In [67]:
graph.run("MATCH (a:Leader) RETURN a.full_name, a.eigenvector").data()

[{'a.full_name': 'Richard (Dick) Sokolsky',
  'a.eigenvector': 0.012195121951219513},
 {'a.full_name': 'Karim Sadjadpour', 'a.eigenvector': 0.09644790978771624},
 {'a.full_name': 'Raz Zimmt', 'a.eigenvector': 0.04735671924036259},
 {'a.full_name': 'Ahmad Majidyar', 'a.eigenvector': 0.057142104769718},
 {'a.full_name': 'Norman T. Roule', 'a.eigenvector': 0.0816877000439798},
 {'a.full_name': 'Pierre Goldschmidt', 'a.eigenvector': 0.012195121951219513},
 {'a.full_name': 'Reuel Mark Gerecht', 'a.eigenvector': 0.012195121951219513},
 {'a.full_name': 'Uzi Rubin', 'a.eigenvector': 0.012195121951219513},
 {'a.full_name': 'Ariane M. Tabatabai', 'a.eigenvector': 0.09644790978771624},
 {'a.full_name': 'Matthew Kroenig', 'a.eigenvector': 0.057142104769718},
 {'a.full_name': 'Afshon Ostovar', 'a.eigenvector': 0.0816877000439798},
 {'a.full_name': 'Gary Samore', 'a.eigenvector': 0.012195121951219513},
 {'a.full_name': 'Ray Takeyh', 'a.eigenvector': 0.012195121951219513},
 {'a.full_name': 'Mark Hibb

In [69]:
graph.run("CALL algo.closeness.harmonic('Leader', 'FOLLOWS', {writeProperty:'centrality_harmonic'}) YIELD nodes,loadMillis, computeMillis, writeMillis")

In [70]:
graph.run("MATCH (a:Leader) RETURN a.full_name, a.centrality_harmonic").data()

[{'a.full_name': 'Richard (Dick) Sokolsky', 'a.centrality_harmonic': 0.0},
 {'a.full_name': 'Karim Sadjadpour',
  'a.centrality_harmonic': 0.42181069958847733},
 {'a.full_name': 'Raz Zimmt', 'a.centrality_harmonic': 0.39300411522633744},
 {'a.full_name': 'Ahmad Majidyar',
  'a.centrality_harmonic': 0.5905349794238683},
 {'a.full_name': 'Norman T. Roule',
  'a.centrality_harmonic': 0.4074074074074074},
 {'a.full_name': 'Pierre Goldschmidt',
  'a.centrality_harmonic': 0.35185185185185186},
 {'a.full_name': 'Reuel Mark Gerecht', 'a.centrality_harmonic': 0.0},
 {'a.full_name': 'Uzi Rubin', 'a.centrality_harmonic': 0.0},
 {'a.full_name': 'Ariane M. Tabatabai',
  'a.centrality_harmonic': 0.47942386831275713},
 {'a.full_name': 'Matthew Kroenig',
  'a.centrality_harmonic': 0.46707818930041145},
 {'a.full_name': 'Afshon Ostovar',
  'a.centrality_harmonic': 0.4773662551440329},
 {'a.full_name': 'Gary Samore', 'a.centrality_harmonic': 0.0},
 {'a.full_name': 'Ray Takeyh', 'a.centrality_harmonic': 

In [74]:
graph.run("MATCH (n),(r) RETURN (n),(r)")